In [1]:
from lonboard import Map, PathLayer, viz
from lonboard.colormap import apply_categorical_cmap
from lonboard.experimental import TripsLayer
import folium
from libpysal.weights import Queen
from libpysal.weights import DistanceBand
from sklearn.preprocessing import MinMaxScaler
import ruptures as rpt  
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import shapely as shp
import hvplot.pandas
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from geopy.distance import geodesic
import geoviews as gv
from IPython.display import display

from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import plotly.express as px
import matplotlib.pyplot as plt
from holoviews import opts, dim
import libpysal
from libpysal.weights import KNN


import warnings
warnings.filterwarnings('ignore')

opts.defaults(opts.Overlay(active_tools=['wheel_zoom'], frame_width=500, frame_height=400))
plot_defaults = {'linewidth':5, 'capstyle':'round', 'figsize':(9,3), 'legend':True}
hvplot_defaults = {'tiles':'CartoLight', 'frame_height':320, 'frame_width':320, 'cmap':'Viridis', 'colorbar':True}
opts.defaults(opts.Overlay(active_tools=['wheel_zoom'], frame_width=500, frame_height=400))
kwargs = {**hvplot_defaults, 'line_width':4}

mpd.show_versions()


MovingPandas 0.20.0

SYSTEM INFO
-----------
python     : 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:20:11) [MSC v.1938 64 bit (AMD64)]
executable : C:\Users\bluer\anaconda3\python.exe
machine    : Windows-11-10.0.26100-SP0

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : None
GEOS lib   : None
GDAL       : 3.9.1
GDAL data dir: C:\Users\bluer\anaconda3\Lib\site-packages\fiona\gdal_data
PROJ       : 9.4.1
PROJ data dir: C:\Users\bluer\anaconda3\Lib\site-packages\pyproj\proj_dir\share\proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 1.0.1
pandas     : 2.2.2
fiona      : 1.10.1
numpy      : 1.26.4
shapely    : 2.0.6
pyproj     : 3.7.0
matplotlib : 3.9.2
mapclassify: 2.8.1
geopy      : 2.4.1
holoviews  : 1.19.1
hvplot     : 0.11.0
geoviews   : 1.14.0
stonesoup  : 1.5


In [2]:
data1 = gpd.read_file(r'C:\Users\bluer\Dropbox\Git-hub\Data\aisdk-2023-11-02(cargo).gpkg')
data = data1.copy()

In [3]:
data2 = pd.read_csv(r'C:\Users\bluer\Dropbox\Git-hub\Data\Max_Speed_CPD_20231102.csv')

In [4]:
merged_data = data.merge(data2, on='MMSI', how='left')
data = merged_data.dropna(subset=['Max Speed(kn)'])

In [5]:
geometry = [Point(xy) for xy in zip(data['Longitude'], data['Latitude'])]
data = gpd.GeoDataFrame(data, geometry=geometry)
data['time'] = pd.to_datetime(data['# Timestamp'], format='%d/%m/%Y %H:%M:%S')
data = data.set_index('time')
collection = mpd.TrajectoryCollection(data, 'MMSI', min_length=10000)
collection

TrajectoryCollection with 357 trajectories

# Preprocess and Adjust Data Range

In [6]:
collection = mpd.MinTimeDeltaGeneralizer(collection).generalize(tolerance=timedelta(minutes=1))

In [7]:
# Preprocessing and Noise Minimization
cleaned = collection.copy()
cleaned = mpd.OutlierCleaner(cleaned).clean(alpha=2)
smoothed = mpd.KalmanSmootherCV(cleaned).smooth(process_noise_std=0.5, measurement_noise_std=1)

In [8]:
for trajectory in smoothed.trajectories:
    trajectory.add_speed(overwrite=True, name="speed (knots)", units=("nm", "h"))
    trajectory.add_timedelta(overwrite=True)
    trajectory.add_distance(overwrite=True, name="distance (m)", units="m")

# Generalize speed before applying PELT

In [9]:
new_df1 = smoothed.to_point_gdf()

In [10]:
new_df1['Speed_Ratio'] = new_df1['speed (knots)'] / new_df1['Max Speed(kn)']

In [11]:
new_df1['time'] = pd.to_datetime(new_df1['# Timestamp'], format='%d/%m/%Y %H:%M:%S')
new_df1 = new_df1.set_index('time')
smoothed = mpd.TrajectoryCollection(new_df1, 'MMSI')
smoothed

TrajectoryCollection with 355 trajectories

# Detect speed changes using PELT

# Visualize change points

In [12]:
new_df1 = smoothed.to_point_gdf()
new_df1['# Timestamp'] = pd.to_datetime(new_df1['# Timestamp'], format='%d/%m/%Y %H:%M:%S')

grouped = new_df1.groupby('MMSI')
change_points_dict = {}

for name, group in grouped:
    test = group['Speed_Ratio'].values
    model = "l1"
    algo = rpt.Pelt(model=model).fit(test)
    penalty_value =1.2 
    my_bkps = algo.predict(pen=penalty_value)
    change_points_dict[name] = my_bkps[:-1] 

threshold = 0.05 

def get_color_change(prev_values, next_values):
    """Function to calculate the direction of change and speed difference"""
    prev_median = np.median(prev_values)
    next_median = np.median(next_values)
    speed_diff = next_median - prev_median

    color = 'red' if speed_diff > 0 else 'blue'
    return color, speed_diff

change_points_data = []
colors = []
speed_diffs = []

for name, group in grouped:
    cp_indices = [0] + change_points_dict[name] + [len(group)]
    for i in range(1, len(cp_indices) - 1):
        prev_values = group['Speed_Ratio'].iloc[cp_indices[i - 1]:cp_indices[i]]
        next_values = group['Speed_Ratio'].iloc[cp_indices[i]:cp_indices[i + 1]]

        color, speed_diff = get_color_change(prev_values, next_values)

        # Apply threshold to remove small changes
        if abs(speed_diff) >= threshold:
            colors.append(color)
            speed_diffs.append(speed_diff)
            change_points_data.append(group.iloc[cp_indices[i] - 1])
        else:
            # Skip change points with differences below the threshol
            continue

# Calculate absolute values of speed differences
abs_speed_diffs = np.abs(speed_diffs)
abs_speed_diffs = np.clip(abs_speed_diffs, None, 0.8) 

# Apply exponential scaling for circle size
size_factor = 50  
min_size = 0 
sizes = (np.exp(abs_speed_diffs*6)) * 10 + min_size

change_points_df = pd.DataFrame(change_points_data)
geometry = [Point(xy) for xy in zip(change_points_df['Longitude'], change_points_df['Latitude'])]
change_points_df = gpd.GeoDataFrame(change_points_df, geometry=geometry, crs='EPSG:4326')

change_points_df['color'] = colors
change_points_df['size'] = sizes

alpha_value = 0.35 

change_points_df.hvplot(geo=True, c='color', size='size', alpha=alpha_value, **kwargs)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (color,size)

# World Port Coordinates Mapping

In [ ]:
World_Port_Index = pd.read_csv(r'C:\Users\bluer\Dropbox\Git-hub\Data\UpdatedPub150.csv', encoding='cp1252')
WPI = pd.DataFrame(World_Port_Index)

geometry = [Point(xy) for xy in zip(WPI['Longitude'], WPI['Latitude'])]

WPI = gpd.GeoDataFrame(WPI, geometry=geometry)

WPI.set_crs(epsg=4326, inplace=True)

In [ ]:
# Function to add arrows and points on the map (considering color and size)
def add_arrows_and_points_on_map(map_object, latitude, longitude, cog, colors, sizes, length=1.2):
    for lat, lon, direction, color, size in zip(latitude, longitude, cog, colors, sizes):
        # Calculate the endpoint of the arrow based on direction and distance
        destination = geodesic(kilometers=length).destination((lat, lon), direction)
        end_lat = destination.latitude
        end_lon = destination.longitude
        
        # Draw arrows on the map (as lines)
        folium.PolyLine([(lat, lon), (end_lat, end_lon)], color=color, weight=2).add_to(map_object)
        
        # Modify the arrowhead as a black triangle (both border and fill color are black)
        folium.RegularPolygonMarker(location=(end_lat, end_lon), fill_color='black', color='black', number_of_sides=3, radius=4, rotation=direction-90).add_to(map_object)
        
        # Add points on the map (considering color and size)
        folium.CircleMarker(location=[lat, lon], radius=size, color=color, fill=True, fill_opacity=0.35).add_to(map_object)
        
        
# Apply a scaling factor to reduce point size
scaling_factor = 0.3  
# scaled_sizes = change_points_df['size'] * scaling_factor
scaled_sizes = ((np.exp(abs_speed_diffs*3)) * 10 + min_size) * scaling_factor

# Create Folium map (set tiles to 'CartoDB positron'
m = folium.Map(location=[change_points_df['Latitude'].mean(), change_points_df['Longitude'].mean()], 
               zoom_start=12, 
               tiles='CartoDB positron',
               control_scale=True)   # Add scale bar

# Convert each trajectory into LineString format and add it to the map
for traj in smoothed.trajectories:
    path_coords = [(coord[1], coord[0]) for coord in traj.to_linestring().coords]  # Extract path coordinates
    folium.PolyLine(
        locations=path_coords,
        color='#D8BFD8',   # Set path color
        weight=2,      # Set path thickness
        opacity=1     # Set path opacity
    ).add_to(m)
    

# Add arrows and points on the map (considering color and scaled size)
add_arrows_and_points_on_map(m, change_points_df['Latitude'], change_points_df['Longitude'], change_points_df['COG'], change_points_df['color'], scaled_sizes)

# Add WPI port data in green
for _, row in WPI.iterrows():
    folium.RegularPolygonMarker(location=[row['Latitude'], row['Longitude']], 
                                number_of_sides=3,  # Triangle shape
                                radius=4,  # Adjust size as needed
                                color='green',   # Border color
                                fill=True, 
                                fill_color='green',  # Fill color
                                fill_opacity=0.8, 
                                rotation=-90,  # Rotate 90 degrees left
                                popup=row['Main Port Name']).add_to(m)

# Add a scale bar based on scaled_sizes values
min_size = min(scaled_sizes)
max_size = max(scaled_sizes)

# Generate scale bar HTML (aligned to the left, centered on circles)
scale_bar_html = f"""
<div style="position: fixed; 
            bottom: 250px; left: 50px; width: 180px; height: auto; 
            background-color: rgba(255, 255, 255, 0.8); z-index: 9999; font-size: 14px; 
            border-radius: 8px; box-shadow: 2px 2px 6px rgba(0,0,0,0.3); 
            padding: 10px; font-family: Arial, sans-serif;">
    <h4 style="margin: 0; text-align: center; font-weight: bold; font-size: 18px;">Size Scale</h4>
    <div style="display: flex; flex-direction: column; align-items: flex-start;">
        <!-- Smallest circle (center-aligned, shifted left) -->
        <div style="display: flex; align-items: center; margin-bottom: 10px;">
            <div style="width: {min_size*1.5:.1f}px; height: {min_size*1.5:.1f}px; border: 2px solid black; border-radius: 50%; background-color: transparent; margin-right: 10px; margin-left: calc(0px + ({max_size*1.5:.1f}px - {min_size*1.5:.1f}px) / 2);"></div>
            <span style="margin-left: 10px;">Gradual Change</span> <!-- Gradual Change -->
        </div>

        <!-- Arrow (vertical, pointing upwards) -->
        <div style="display: flex; align-items: center; justify-content: center; width: 100%; margin-bottom: 10px;">
            <div style="border-left: 2px solid black; height: 50px; position: relative;">
                <div style="position: absolute; top: 40px; right: -4.2px; border-top: 10px solid black; border-left: 5px solid transparent; border-right: 5px solid transparent;"></div>
            </div>
        </div>

        <!-- Largest circle (aligned to the left border) -->
        <div style="display: flex; align-items: center;">
            <div style="width: {max_size*1.3:.1f}px; height: {max_size*1.3:.1f}px; border: 2px solid black; border-radius: 50%; background-color: transparent; margin-right: 10px; margin-left: 0px;"></div>
            <span>Sudden Change</span> <!-- Sudden Change -->
        </div>
    </div>
</div>
"""

m.get_root().html.add_child(folium.Element(scale_bar_html))

legend_html = """
<div style="position: fixed; 
            bottom: 100px; left: 50px; width: 180px; height: auto; 
            background-color: rgba(255, 255, 255, 0.8); z-index: 9999; font-size: 14px; 
            border-radius: 8px; box-shadow: 2px 2px 6px rgba(0,0,0,0.3); 
            padding: 10px; line-height: 1.2; font-family: Arial, sans-serif;">
    <h4 style="margin: 0; text-align: center; font-weight: bold; font-size: 18px;">Legend</h4>
    <p style="margin: 5px 0; font-size: 14px;">
        <i style="background: blue; border-radius: 50%; width: 14px; height: 14px; display: inline-block; margin-right: 10px;"></i>
        Speed Decrease
    </p>
    <p style="margin: 5px 0; font-size: 14px;">
        <i style="background: red; border-radius: 50%; width: 14px; height: 14px; display: inline-block; margin-right: 10px;"></i>
        Speed Increase
    </p>
    <p style="margin: 5px 0; font-size: 14px;">
        <i style="border: none; width: 0; height: 0; border-left: 7px solid transparent; border-right: 7px solid transparent; border-bottom: 14px solid green; display: inline-block; margin-right: 10px; transform: rotate(0deg);"></i>
        Port
    </p>
    <p style="margin: 5px 0; font-size: 14px;">
        <i style="border: none; width: 0; height: 0; border-left: 7px solid transparent; border-right: 7px solid transparent; border-bottom: 14px solid black; display: inline-block; margin-right: 10px; transform: rotate(0deg);"></i>
        Direction
    </p>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

scale_bar_css = '''
<style>
/* Scale bar */
.leaflet-control-scale {
    font-size: 14px;  /* Adjust font size (default is about 10px) */
    left: 50px !important;  /* Set the scale bar 50px from the left */
    bottom: 20px !important;  /* Move the scale bar 20px up from the bottom */
}


/* Scale bar line */
.leaflet-control-scale-line {
    border-top: 2px solid #777;  /* Adjust line thickness (default is 1px) */
}
</style>
'''
m.get_root().html.add_child(folium.Element(scale_bar_css))

# Save the map
m.save('Detection Results of Speed Increase and Decrease Zones.html')

m

In [16]:
change_points_df1 = change_points_df.copy()

coordinates = change_points_df['geometry'].apply(lambda geom: (geom.x, geom.y))

# Creating and Fitting a DBSCAN Model
dbscan_model = DBSCAN(eps=0.15, min_samples=60)
clusters = dbscan_model.fit_predict(list(coordinates))

change_points_df1['cluster'] = clusters
change_points_df1 = change_points_df1[change_points_df1['cluster'] != -1]
change_points_df1.hvplot(geo=True, c='color', size='size', alpha=alpha_value, **kwargs)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (color,size)

In [ ]:
def add_arrows_and_points_on_map(map_object, latitude, longitude, cog, colors, sizes, length=1.2):
    for lat, lon, direction, color, size in zip(latitude, longitude, cog, colors, sizes):
        destination = geodesic(kilometers=length).destination((lat, lon), direction)
        end_lat = destination.latitude
        end_lon = destination.longitude
            
        folium.PolyLine([(lat, lon), (end_lat, end_lon)], color=color, weight=2).add_to(map_object)
        
        folium.RegularPolygonMarker(location=(end_lat, end_lon), fill_color='black', color='black', number_of_sides=3, radius=4, rotation=direction-90).add_to(map_object)
        
        folium.CircleMarker(location=[lat, lon], radius=size, color=color, fill=True, fill_opacity=0.35).add_to(map_object)
        
scaling_factor = 0.3  
scaled_sizes = ((np.exp(abs_speed_diffs*3)) * 10 + min_size) * scaling_factor

m = folium.Map(location=[change_points_df1['Latitude'].mean(), change_points_df1['Longitude'].mean()], 
               zoom_start=12, 
               tiles='CartoDB positron',
               control_scale=True) 

for traj in smoothed.trajectories:    
    path_coords = [(coord[1], coord[0]) for coord in traj.to_linestring().coords] 
    folium.PolyLine(
        locations=path_coords,
        color='#D8BFD8', 
        weight=2, 
        opacity=1 
    ).add_to(m)
    
add_arrows_and_points_on_map(m, change_points_df1['Latitude'], change_points_df1['Longitude'], change_points_df1['COG'], change_points_df1['color'], scaled_sizes)

for _, row in WPI.iterrows():
    folium.RegularPolygonMarker(location=[row['Latitude'], row['Longitude']], 
                                number_of_sides=3,
                                radius=4, 
                                color='green',  
                                fill=True, 
                                fill_color='green', 
                                fill_opacity=0.8, 
                                rotation=-90,
                                popup=row['Main Port Name']).add_to(m)

min_size = min(scaled_sizes)
max_size = max(scaled_sizes)

scale_bar_html = f"""
<div style="position: fixed; 
            bottom: 250px; left: 50px; width: 180px; height: auto; 
            background-color: rgba(255, 255, 255, 0.8); z-index: 9999; font-size: 14px; 
            border-radius: 8px; box-shadow: 2px 2px 6px rgba(0,0,0,0.3); 
            padding: 10px; font-family: Arial, sans-serif;">
    <h4 style="margin: 0; text-align: center; font-weight: bold; font-size: 18px;">Size Scale</h4>
    <div style="display: flex; flex-direction: column; align-items: flex-start;">
        <div style="display: flex; align-items: center; margin-bottom: 10px;">
            <div style="width: {min_size*1.5:.1f}px; height: {min_size*1.5:.1f}px; border: 2px solid black; border-radius: 50%; background-color: transparent; margin-right: 10px; margin-left: calc(0px + ({max_size*1.5:.1f}px - {min_size*1.5:.1f}px) / 2);"></div>
            <span style="margin-left: 10px;">Gradual Change</span> <!-- Gradual Change -->
        </div>
        <div style="display: flex; align-items: center; justify-content: center; width: 100%; margin-bottom: 10px;">
            <div style="border-left: 2px solid black; height: 50px; position: relative;">
                <div style="position: absolute; top: 40px; right: -4.2px; border-top: 10px solid black; border-left: 5px solid transparent; border-right: 5px solid transparent;"></div>
            </div>
        </div>
        <div style="display: flex; align-items: center;">
            <div style="width: {max_size*1.3:.1f}px; height: {max_size*1.3:.1f}px; border: 2px solid black; border-radius: 50%; background-color: transparent; margin-right: 10px; margin-left: 0px;"></div>
            <span>Sudden Change</span> <!-- Sudden Change -->
        </div>
    </div>
</div>
"""

m.get_root().html.add_child(folium.Element(scale_bar_html))

legend_html = """
<div style="position: fixed; 
            bottom: 100px; left: 50px; width: 180px; height: auto; 
            background-color: rgba(255, 255, 255, 0.8); z-index: 9999; font-size: 14px; 
            border-radius: 8px; box-shadow: 2px 2px 6px rgba(0,0,0,0.3); 
            padding: 10px; line-height: 1.2; font-family: Arial, sans-serif;">
    <h4 style="margin: 0; text-align: center; font-weight: bold; font-size: 18px;">Legend</h4>
    <p style="margin: 5px 0; font-size: 14px;">
        <i style="background: blue; border-radius: 50%; width: 14px; height: 14px; display: inline-block; margin-right: 10px;"></i>
        Speed Decrease
    </p>
    <p style="margin: 5px 0; font-size: 14px;">
        <i style="background: red; border-radius: 50%; width: 14px; height: 14px; display: inline-block; margin-right: 10px;"></i>
        Speed Increase
    </p>
    <p style="margin: 5px 0; font-size: 14px;">
        <i style="border: none; width: 0; height: 0; border-left: 7px solid transparent; border-right: 7px solid transparent; border-bottom: 14px solid green; display: inline-block; margin-right: 10px; transform: rotate(0deg);"></i>
        Port
    </p>
    <p style="margin: 5px 0; font-size: 14px;">
        <i style="border: none; width: 0; height: 0; border-left: 7px solid transparent; border-right: 7px solid transparent; border-bottom: 14px solid black; display: inline-block; margin-right: 10px; transform: rotate(0deg);"></i>
        Direction
    </p>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

scale_bar_css = '''
<style>
.leaflet-control-scale {
    font-size: 14px;
    left: 50px !important;
    bottom: 20px !important;
}

.leaflet-control-scale-line {
    border-top: 2px solid #777;
}
</style>
'''
m.get_root().html.add_child(folium.Element(scale_bar_css))


m.save('Detection Results of Major Speed Increase and Decrease Zones.html')

m